In [1]:
# !pip install gdal

In [2]:
  # !apt-get update && apt-get install -y --no-install-recommends libgdal-dev gdal-bin
  # %pip install --no-cache-dir --force-reinstall "gdal==3.10.3"

In [3]:
from osgeo import gdal

# This will print the installed GDAL version
print(gdal.VersionInfo())

3120000


In [4]:
!gdalinfo --version

GDAL 3.12.0 "Chicoutimi", released 2025/11/03


In [5]:
from dotenv import load_dotenv
load_dotenv()


True

In [6]:
!pip install gcsfs

In [7]:
import os
import time
import xarray as xr
from numcodecs import Blosc
import gcsfs

# Re-use a global filesystem client when possible
gcs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')

In [8]:
import os
from google.cloud import storage

# 2️⃣ Initialize GCS client
client = storage.Client.from_service_account_json('/usr/src/app/user_id.json')

# 3️⃣ List buckets (to verify connection)
for bucket in client.list_buckets():
    print("✅ Found bucket:", bucket.name)

✅ Found bucket: esa_treeo_wfs_backup
✅ Found bucket: remote-sensing-treeo
✅ Found bucket: remote_sensing_saas


In [9]:
bucket_name = "remote_sensing_saas"
bucket = client.bucket(bucket_name)

blobs = list(bucket.list_blobs(prefix=""))  # list all files
for blob in blobs:  # show only first 10
    if '.tif' in blob.name:
        print(blob.name)

01-korindo/00-drone/COG_ORTHO_PT_STL_KALBAR_Blok_6_TIFF.tif
01-korindo/00-drone/COG_ORTHO_PT_STL_KALBAR_PINOH_SELATAN_TIFF.tif
01-korindo/00-drone/ORTHO_PT_STL_KALBAR_Blok 1_TIFF_COG_rev.tif
01-korindo/00-drone/ORTHO_PT_STL_KALBAR_Blok 3_TIFF.tif_COG.tif
01-korindo/00-drone/ORTHO_PT_STL_KALBAR_Blok 4_TIFF_COG.tif
01-korindo/00-drone/ORTHO_PT_STL_KALBAR_Blok 5_TIFF_COG_rev_2.tif
01-korindo/00-drone/ORTHO_PT_STL_KALBAR_Blok_2_TIFF_COG.tif
01-korindo/00-drone/ORTHO_PT_STL_KALBAR_TENGKAJAU_TIFF_COG.tif
01-korindo/00-drone/Ortho High_PT.STL/BLOK 3/ORTHO_PT_STL_KALBAR_Blok 3_TIFF.tif
01-korindo/00-drone/Ortho High_PT.STL/BLOK 4/ORTHO_PT_STL_KALBAR_Blok 4_TIFF.tif
01-korindo/00-drone/Ortho High_PT.STL/BLOK 5/ORTHO_PT_STL_KALBAR_Blok 5_TIFF.tif
01-korindo/00-drone/Ortho High_PT.STL/BLOK 6a/ORTHO_PT_STL_KALBAR_Blok 6_TIFF.tif
01-korindo/00-drone/Ortho High_PT.STL/BLOK 6b/ORTHO_PT_STL_KALBAR_PINOH_SELATAN_TIFF.tif
01-korindo/00-drone/Ortho High_PT.STL/Logpond/ORTHO_PT_STL_KALBAR_TENGKAJAU_TIFF.t

In [10]:
import subprocess

src_vsigs = "/vsigs/remote_sensing_saas/01-korindo/00-drone/Ortho High_PT.STL/BLOK 4/ORTHO_PT_STL_KALBAR_Blok 4_TIFF.tif"

# Run the command and capture output
# Note: Using a list like ['gdalinfo', src_vsigs] automatically handles spaces in the path
result = subprocess.run(['gdalinfo', src_vsigs], capture_output=True, text=True, check=True)

# The output is now in a Python variable!
info_text = result.stdout
print(info_text)

Driver: GTiff/GeoTIFF
Files: /vsigs/remote_sensing_saas/01-korindo/00-drone/Ortho High_PT.STL/BLOK 4/ORTHO_PT_STL_KALBAR_Blok 4_TIFF.tif
Size is 260057, 323688
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 49S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 49S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",111,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["Fal

In [11]:
!ls -lah /mnt/data/

total 89M
drwxr-xr-x 2 root root 4.0K Nov 11 06:14  .
drwxr-xr-x 1 root root 4.0K Nov 10 03:14  ..
-rw-r--r-- 1 root root  89M Nov 11 06:14 'ORTHO_PT_STL_KALBAR_Blok 4_TIFF_rev.tif'


In [13]:
### REMOVE THE DATA - CONFIRM ITS ALREADY DOWNLOADED
# !rm /mnt/data/*

In [14]:
# src = src_vsigs
# dst = f'/mnt/data/{os.path.basename(src)[:-4]}_rev.tif'
# dst

In [ ]:
### FOR LOWER SPEC
import os
import time
from osgeo import gdal

# --- GDAL Cloud Storage Fix (Crucial!) ---
os.environ["CPL_VSIL_USE_TEMP_FILE_FOR_RANDOM_WRITE"] = "YES"

# --- 1. SET TEMP DIRECTORY ---
# CRITICAL: You must MANUALLY CREATE this folder first!
# Go to C:\ and create a new folder named "temp".
os.environ["CPL_TMPDIR"] = '/mnt/data'

# --- OPTIMIZE FOR LOW RAM (8GB) ---
gdal.SetConfigOption('GDAL_CACHEMAX', '2048')

# --- CONFIG ---
# src = "/vsigs/remote_sensing_saas/01-korindo/00-drone/ORTHO_PT_STL_KALBAR_Blok 5_TIFF_COG.tif"
# dst = '/mnt/data/ORTHO_PT_STL_KALBAR_Blok 5_TIFF_COG_rev_2.tif'

src = src_vsigs
dst = f'/mnt/data/{os.path.basename(src)[:-4]}_COG_rev.tif'
# dst


# --- START TIMER ---
start = time.time()

# --- DEFINE PROGRESS CALLBACK ---
def progress_callback(complete, message, unknown):
    percent = int(complete * 100)
    print(f"\rProgress (local creation): {percent}% | {message}      ", end="", flush=True)

# --- 2. RUN TRANSLATE ---
print(f"🚀 Starting GDAL COG translation...\nInput: {src}\nOutput: {dst}\n")

translate_options = [
        "COMPRESS=ZSTD",
        "BIGTIFF=YES",
        "PREDICTOR=2",
        "BLOCKSIZE=512",
        "OVERVIEWS=AUTO",
        "NUM_THREADS=ALL_CPUS"
    ]

result = gdal.Translate(
    dst,
    src,
    format="COG",
    creationOptions=translate_options,
    callback=progress_callback
)

# Ensure we print a newline after the progress bar
print() 

# --- 3. THIS IS THE CORRECTED PART ---
if result is not None:
    print("✅ Local COG creation complete.")
    print("⏳ Forcing upload to Google Cloud Storage... (This may take a while)")
    
    # --- THIS IS THE FIX ---
    # Force GDAL to write all data and close the file handle.
    # This blocks the script until the upload is 100% finished.
    result.FlushCache()  
    result = None        # This line triggers the actual upload
    # --- END FIX ---
    
    # NOW we can safely stop the timer
    end = time.time()
    elapsed = end - start
    print(f"✅ Upload complete! Total time: {elapsed:.2f} seconds.")
    print("\n--- Running verification... ---")
    
    # --- 4. IMMEDIATE VERIFICATION ---
    gdal.PushErrorHandler('CPLQuietErrorHandler') 
    ds_verify = gdal.Open(dst)
    gdal.PopErrorHandler() 

    if ds_verify is not None:
        print("✅✅✅ SUCCESS! File is confirmed in the bucket.")
        ds_verify = None
    else:
        print("❌❌❌ FAILURE! Verification step failed. File not found in bucket.")

else:
    # This means gdal.Translate failed locally
    end = time.time()
    elapsed = end - start
    print(f"❌ GDAL translate failed during *local* creation after {elapsed:.2f} seconds.")

🚀 Starting GDAL COG translation...
Input: /vsigs/remote_sensing_saas/01-korindo/00-drone/Ortho High_PT.STL/BLOK 4/ORTHO_PT_STL_KALBAR_Blok 4_TIFF.tif
Output: /mnt/data/ORTHO_PT_STL_KALBAR_Blok 4_TIFF_COG_rev.tif



/opt/venv/lib/python3.12/site-packages/osgeo/gdal.py:606: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Progress (local creation): 34% |       

In [16]:
print('test')

test


In [ ]:
# !rm -rf /mnt/data/*

In [ ]:
### UPLOADING TO GCS
import os
from google.cloud import storage
from google.oauth2 import service_account
from tqdm.notebook import tqdm

gcs = gcsfs.GCSFileSystem(
    project=os.getenv("GOOGLE_CLOUD_PROJECT"), 
    token='/usr/src/app/user_id.json',
    access='full_control'
)

def upload_file_to_gcs_with_progress(bucket_name, source_file_path, destination_blob_name):
    """
    Uploads a local file to GCS with a progress bar in Jupyter.
    
    Args:
        bucket_name (str): The name of your GCS bucket.
        source_file_path (str): The local path to the file you want to upload.
        destination_blob_name (str): The name you want the file to have in GCS.
    """
    # 1. Load credentials from the service account file
    credentials = service_account.Credentials.from_service_account_file(
        '/usr/src/app/user_id.json',
        scopes=['https://www.googleapis.com/auth/cloud-platform']
    )
    
    # 2. Instantiate the GCS client with explicit credentials
    storage_client = storage.Client(
        project=os.getenv("GOOGLE_CLOUD_PROJECT"),
        credentials=credentials
    )
    
    # 3. Get the target bucket
    try:
        bucket = storage_client.bucket(bucket_name)
    except Exception as e:
        print(f"Error getting bucket '{bucket_name}': {e}")
        return
    
    # 4. Create a new blob (file) in the bucket
    blob = bucket.blob(destination_blob_name)
    
    # 5. Get the total file size for the progress bar
    try:
        file_size = os.path.getsize(source_file_path)
    except FileNotFoundError:
        print(f"Error: The file {source_file_path} was not found.")
        return
    except Exception as e:
        print(f"Error getting file size: {e}")
        return
    
    # 6. Upload the file using upload_from_file and the tqdm wrapper
    try:
        with open(source_file_path, "rb") as f:
            with tqdm.wrapattr(f, "read", total=file_size, desc="Uploading", unit="B", unit_scale=True, unit_divisor=1024) as file_with_progress:
                blob.upload_from_file(file_with_progress, size=file_size)
        
        print(f"\n✅ Upload complete: {source_file_path} to {bucket_name}/{destination_blob_name}.")
    
    except Exception as e:
        print(f"\nError during upload: {e}")

# --- How to use the function ---
MY_BUCKET_NAME = bucket_name
GCS_DESTINATION_NAME = f"01-korindo/00-drone/{os.path.basename(dst)}"

# Call the function
upload_file_to_gcs_with_progress(MY_BUCKET_NAME, dst, GCS_DESTINATION_NAME)

In [ ]:
os.path.basename(dst)